Goal:
Create a MongoDB database of every unique team in the Kaggle Football Delphi database. 

Include: Team name, 2011 total goals, 2011 total wins, histogram of team's wins and losses, Rainy Day Win Percentage (proxy rain in Berlin, Germany) in 2011

MongoDB is a document structure with collecions of documents. 
Each document is a dictionary:
{ key1: entry1, key2: entry3..}
where each entry can be any type: strings, integers, lists, lists of other documents

In [1]:
import sqlite3
import pandas as pd
connection = sqlite3.connect('database.sqlite')
c = connection.cursor()

In [3]:
# Tables
c.execute('select name from sqlite_master where type = "table"').fetchall()

In [4]:
# Columns for Respective Tables
for table in c.execute('select name from sqlite_master where type = "table"').fetchall():
    val = table[0]
    c.execute(f"select * from {val}")
    print([x[0] for x in c.description])

In [1]:
# Matches: Match_ID, Div, Season, Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR
# pd.DataFrame(c.execute("select * from Matches where Season == 2011 and (HomeTeam = 'Bayern Munic' or AwayTeam == 'Bayern Munich')").fetchall(),columns = [x[0] for x in c.description])

# Teams_in_Matches: Match_ID, Unique_Team_ID
# two rows per match, listing each team ID
# pd.DataFrame(c.execute("select * from Teams_in_Matches").fetchall(),columns = [x[0] for x in c.description])

# Teams: Season, TeamName, KaderHome, AvgAgeHome, ForeignPlayersHome, OverallMarketValueHome, AvgMarketValueHome, StadiumCapacity
# rows include Each Team, Season
#  pd.DataFrame(c.execute("select * from Teams where Season == 2011").fetchall(),columns = [x[0] for x in c.description])

# Unique Teams: TeamName, Unique_Team_ID
# pd.DataFrame(c.execute("select * from Unique_Teams").fetchall(),columns = [x[0] for x in c.description])

In [6]:
# Index of our Final Table
teams = pd.DataFrame(c.execute("""select TeamName, Unique_Team_ID from Teams join Unique_Teams using(TeamName) where Season == 2011 order by TeamName""").fetchall(),columns = [x[0] for x in c.description])

In [12]:
# Retrieve Weather Data from Match Days

# We only need to know if it rained the day the team played. 
date_series = pd.DataFrame(c.execute("""select Date 
                        from Matches
                        where Season == 2011
                        order by Date""").fetchall(),columns = [x[0] for x in c.description])

date_series = date_series.Date.unique() # returns array of strings

In [30]:
# Convert to DateTime
from datetime import datetime
date_datetime = pd.to_datetime(date_series, format='%Y-%m-%d') # this is local timezone
epoch_time = list((date_datetime.unique() - datetime(1970,1,1,1,0,0)).total_seconds()) # start of epoch time in Germany, shouldn't really matter except to shift the 24 hour window that we look for rain
epoch_time = [int(x) for x in epoch_time]

165

In [85]:
# Merge in Weather Data
import requests
import dark_sky_key

# UNIT TEST: Berlin Coordinates {+52.5200° N, + 13.4050° E}
time = int(epoch_time[0])
dark_sky_url = f'https://api.darksky.net/forecast/{dark_sky_key.key}/52.5200,13.4050,{time}?exclude=currently,hourly'
response = requests.get(dark_sky_url)

In [255]:
# Call DarkSky API for data or see next cell to load pickle/CSV

# rain_data = []
# for i, game in enumerate(epoch_time):
#     time = int(game)
#     dark_sky_url = f'https://api.darksky.net/forecast/{dark_sky_key.key}/52.5200,13.4050,{time}?exclude=currently,hourly'
#     #response = requests.get(dark_sky_url)
#     if response.status_code == 200:
#         doc = response.json()
#         rain_day = {'precipIntensity':doc['daily']['data'][0]['precipIntensity'],
#                     'precipIntensityMax':doc['daily']['data'][0]['precipIntensityMax'],
#                     'precipProbability':doc['daily']['data'][0]['precipProbability']
#                    }
#         rain_data.append(rain_day)
#     else:
#         break
        
# # Save to CSV
# pd.DataFrame(rain_data).to_csv('DarkSkyRainDataShort.csv')

# # Pickle
# import pickle
# with open('historicalRainDarkSkyAPIshort.pickle', 'wb') as f:
#     pickle.dump(rain_data, f)

In [37]:
# Load from Pickle
import pickle
file = open("historicalRainDarkSkyAPI.pickle",'rb')
rain_data = pickle.load(file)

In [44]:
# Provide Date Primary Key for Rain Data
weather = pd.DataFrame()
weather['Date'] = date_datetime

# Clean Up Rain Data (drop duplicates if you want)
# Generate Rain Mask
final = pd.concat([weather,pd.DataFrame(rain_data)],axis = 1)
final['Rain'] = [1 if x > 0 else 0 for x in final.precipIntensityMax ]
final.drop(['precipIntensity', 'precipIntensityMax', 'precipProbability'],axis=1,inplace = True)
final.reset_index(inplace = True)
final.drop('index',axis=1,inplace=True)

final['Date'] = [str(x)[0:10] for x in final.Date]

992

In [45]:
# Convert to Tuples for SQLite Table
arg_tuples = [tuple(x) for x in final.values]

In [47]:
# Add Rain Data to SQLite
c.execute("""create table match_rain (Date TEXT PRIMARY KEY,
                                      Rain INTEGER);""")

c.executemany("insert into match_rain values (?,?)",arg_tuples)
connection.commit()

In [64]:
# Merge this in Pandas to make sure it looks ok
matches = pd.DataFrame(c.execute("select * from Matches where Season == 2011").fetchall(),columns = [x[0] for x in c.description])
matches.merge(final,on='Date').sort_values(by = 'Date')
# looks good

In [71]:
pd.DataFrame(c.execute("""select * from Matches join match_rain using(Date)""").fetchall(),columns = [s[0] for s in c.description]).head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Rain
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,1
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,0
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,0
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,0
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,0


In [72]:
# New Tables for Team's Home and Away Performances in 2011
home_stats = c.execute("""select HomeTeam, sum(FTHG) as HomeGoals, sum(FTR=='H') as HomeWins, Sum(Rain) as HomeRain, Sum(Rain and FTR=='H') as HomeRainyWins
                        from Matches
                        join match_rain using(Date)
                        group by HomeTeam, Season
                        having Season == 2011""").fetchall()

away_stats = c.execute("""select AwayTeam, sum(FTAG) as AwayGoals, sum(FTR=='A') as AwayWins, Sum(Rain) as AwayRain, Sum(Rain and FTR=='A') as AwayRainyWins
                        from Matches                        
                        join match_rain using(Date)
                        group by AwayTeam, Seasonbb
                        having Season == 2011""").fetchall()

In [84]:
# Home and Away Tables in Database to merge on to 2011 Teams
c.execute("""create table home_stats (TeamName TEXT PRIMARY KEY,
                                      HomeGoals INTEGER,
                                      HomeWins INTEGER,
                                      HomeRainGames INTEGER,
                                      HomeRainWins INTEGER);""")
c.executemany("insert into home_stats values (?,?,?,?,?)",home_stats)

c.execute("""create table away_stats (TeamName PRIMARY KEY,
                                      AwayGoals INTEGER,
                                      AwayWins INTEGER,
                                      AwayRainGames INTEGER,
                                      AwayRainWins INTEGER)""")
c.executemany("insert into away_stats values (?,?,?,?,?)",away_stats)
connection.commit()

In [137]:
# Merge
test = pd.DataFrame(c.execute("""select TeamName, Unique_Team_ID, HomeGoals+AwayGoals as TotalGoals, HomeWins + AwayWins as Wins, ((HomeRainWins + AwayRainWins) *100) / (HomeRainGames+AwayRainGames) as RainyWinPerc
                        from Teams 
                        join Unique_Teams using(TeamName) 
                        join home_stats using(TeamName)
                        join away_stats using(TeamName)
                        where Season == 2011 order by TeamName""").fetchall(),columns = [x[0] for x in c.description])

In [151]:
class MongoEntry():
    def __init__(self,name=''):
        self.name = name
    def to_dict(self,data):
        if type(data) == pd.core.frame.DataFrame:
            self.data = test.to_dict(orient = 'records')
        if type(data) == list and type(data[0]) == tuple:
            cols = ['TeamName','Unique_Team_ID','TotalGoals','Wins','RainyWinPerc']
            self.data = pd.DataFrame(data,columns = cols).to_dict(orient = 'records')  

In [152]:
mongo = MongoEntry()
mongo.to_dict(test)

In [155]:
# Final Step
import pymongo
local_instance = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
delphi_db = local_instance['DelphiFootball2011']
delphi_collection = delphi_db['Season2011']

# recall this datbase does not exist in the server until you populate it

insert_result = delphi_collection.insert_many(mongo.data)

In [ ]:
# What sort of histogram do they want? Need uncollapsed data. Goals in wins/losses? Rain or not rain?